# Gotobi Strategy Backtest Demo

Demonstrates running the Gotobi FX settlement strategy using NautilusTrader's BacktestEngine.
Loads USDJPY data from parquet files and runs both the basic Gotobi and stop-loss variants.

In [1]:
from pathlib import Path
import sys
import importlib

import pandas as pd
import plotly.graph_objects as go

from nautilus_trader.backtest.engine import BacktestEngineConfig
from nautilus_trader.common.config import LoggingConfig
from nautilus_trader.model.data import BarType


def find_project_root(start: Path) -> Path:
    for candidate in (start, *start.parents):
        if (candidate / "pyproject.toml").exists() and (candidate / "trader").exists():
            return candidate
    raise RuntimeError("Could not find project root containing pyproject.toml and trader/.")


PROJECT_ROOT = find_project_root(Path.cwd().resolve())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from trader import DataHandler, SIM

import trader.data.catalog as catalog
importlib.reload(catalog)
dataframe_to_nautilus_bars = catalog.dataframe_to_nautilus_bars

import trader.core.instruments as instruments
importlib.reload(instruments)
make_fx_pair = instruments.make_fx_pair

import trader.config.node as node
importlib.reload(node)
build_backtest_engine = node.build_backtest_engine

import trader.strategy.gotobi as gotobi
importlib.reload(gotobi)
GotobiStrategy = gotobi.GotobiStrategy
GotobiConfig = gotobi.GotobiConfig
GotobiWithSLStrategy = gotobi.GotobiWithSLStrategy
GotobiWithSLConfig = gotobi.GotobiWithSLConfig

print(f"Project root: {PROJECT_ROOT}")


Project root: /Users/justin/Algo-Trading


## Load Data

In [2]:
# Load USDJPY parquet data

data_candidates = [
    PROJECT_ROOT / "data" / "USDJPY_1min_2024-01-01_2025-10-01.parquet",
    PROJECT_ROOT / "data" / "usdjpy_1min_2024-01-01_2025-10-01.parquet",
]
data_path = next((path for path in data_candidates if path.exists()), None)
if data_path is None:
    raise FileNotFoundError(f"Could not find USDJPY parquet in: {data_candidates}")

handler = DataHandler()
df = handler.load_parquet(str(data_path), tz="Asia/Tokyo")

print(f"Loaded {len(df):,} bars from {df.index[0]} to {df.index[-1]}")
print(f"Data path: {data_path}")


Loaded 644,043 bars from 2024-01-01 09:32:00+09:00 to 2025-10-01 08:59:00+09:00
Data path: /Users/justin/Algo-Trading/data/USDJPY_1min_2024-01-01_2025-10-01.parquet


## Build Instruments and Bar Data

In [3]:
# Create USDJPY instrument
instrument = make_fx_pair("USDJPY", SIM, lot_size=100_000)

# Define bar type
bar_type = BarType.from_str(f"{instrument.id}-1-MINUTE-MID-EXTERNAL")

# Convert DataFrame to NautilusTrader bars
bars = dataframe_to_nautilus_bars(df, bar_type, price_precision=instrument.price_precision)
print(f"Converted {len(bars):,} bars for {bar_type}")


Converted 644,043 bars for USD/JPY.SIM-1-MINUTE-MID-EXTERNAL


## Run Gotobi Strategy (No Stop Loss)

In [4]:
# Configure strategies
long_config = GotobiConfig(
    instrument_id=str(instrument.id),
    bar_type=str(bar_type),
    entry_time="09:00:00",
    exit_time="09:55:00",
    trade_size=50.0,
    use_holidays=True,
)

short_config = GotobiConfig(
    instrument_id=str(instrument.id),
    bar_type=str(bar_type),
    entry_time="10:00:00",
    exit_time="10:45:00",
    trade_size=-50.0,
    use_holidays=True,
)

long_strat = GotobiStrategy(config=long_config)
short_strat = GotobiStrategy(config=short_config)

venue_currency = instrument.quote_currency.code
starting_balance = 150_000


def build_engine(strategies):
    return build_backtest_engine(
        instruments=[instrument],
        bars={bar_type: bars},
        strategies=strategies,
        venue=SIM,
        venue_currency=venue_currency,
        starting_balance=starting_balance,
        leverage=500.0,
        config=BacktestEngineConfig(
            logging=LoggingConfig(log_level="ERROR", log_colors=False),
        ),
    )


# Build and run engine
engine = build_engine([long_strat, short_strat])
engine.run()
print(f"Backtest complete. Analysis currency: {venue_currency}")


Backtest complete. Analysis currency: JPY


## Run Gotobi With Stop Loss

In [18]:
# Reset and run with stop-loss variant
long_sl_config = GotobiWithSLConfig(
    instrument_id=str(instrument.id),
    bar_type=str(bar_type),
    entry_time="09:00:00",
    exit_time="09:55:00",
    trade_size=50.0,
    stop_loss_pct=0.05,
    use_holidays=True,
)

short_sl_config = GotobiWithSLConfig(
    instrument_id=str(instrument.id),
    bar_type=str(bar_type),
    entry_time="10:00:00",
    exit_time="10:45:00",
    trade_size=-50.0,
    stop_loss_pct=0.05,
    use_holidays=True,
)

long_sl = GotobiWithSLStrategy(config=long_sl_config)
short_sl = GotobiWithSLStrategy(config=short_sl_config)

engine_sl = build_engine([long_sl, short_sl])
engine_sl.run()
print("Stop-loss backtest complete.")


Stop-loss backtest complete.


In [19]:
# # Test different stop loss percentages
# stop_loss_values = [0.005, 0.01, 0.02, 0.03, 0.05, 0.075, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# results = []

# print(f"Testing {len(stop_loss_values)} stop loss values...")
# for sl_pct in stop_loss_values:
#     # Configure strategies with current stop loss
#     long_test_config = GotobiWithSLConfig(
#         instrument_id=str(instrument.id),
#         bar_type=str(bar_type),
#         entry_time="09:00:00",
#         exit_time="09:55:00",
#         trade_size=50.0,
# #         stop_loss_pct=sl_pct,
#         use_holidays=True,
#     )
    
#     short_test_config = GotobiWithSLConfig(
#         instrument_id=str(instrument.id),
#         bar_type=str(bar_type),
#         entry_time="10:00:00",
#         exit_time="10:45:00",
#         trade_size=-50.0,
# #         stop_loss_pct=sl_pct,
#         use_holidays=True,
#     )
    
#     long_test = GotobiWithSLStrategy(config=long_test_config)
#     short_test = GotobiWithSLStrategy(config=short_test_config)
    
#     # Run backtest
#     engine_test = build_engine([long_test, short_test])
#     engine_test.run()
    
#     # Get performance metrics
#     analyzer = engine_test.portfolio.analyzer
#     pnl_stats = analyzer.get_performance_stats_pnls(currency=analysis_currency)
#     returns_stats = analyzer.get_performance_stats_returns()
#     general_stats = analyzer.get_performance_stats_general()
    
#     # Store results
#     results.append({
#         'stop_loss_pct': sl_pct,
#         'stop_loss_bps': sl_pct * 10000,  # basis points
#         'total_pnl': pnl_stats.get('PnL (total)', 0),
#         'sharpe_ratio': returns_stats.get('Sharpe Ratio', 0),
#         'max_drawdown': returns_stats.get('Max Drawdown', 0),
#         'win_rate': general_stats.get('Win Rate', 0),
#         'total_trades': general_stats.get('Total trades', 0),
#     })
    
#     print(f"SL {sl_pct*10000:.1f} bps: PnL={pnl_stats.get('PnL (total)', 0):,.2f} {analysis_currency_code}, Sharpe={returns_stats.get('Sharpe Ratio', 0):.3f}")

# # Create results DataFrame
# results_df = pd.DataFrame(results).set_index('stop_loss_bps')
# print("\nOptimization complete!")


In [20]:
# # Display results sorted by different metrics
# print("Top 5 by Total PnL:")
# print(results_df.sort_values('total_pnl', ascending=False)[['total_pnl', 'sharpe_ratio', 'max_drawdown', 'win_rate']].head())

# print("\nTop 5 by Sharpe Ratio:")
# print(results_df.sort_values('sharpe_ratio', ascending=False)[['total_pnl', 'sharpe_ratio', 'max_drawdown', 'win_rate']].head())

# print("\nTop 5 by Win Rate:")
# print(results_df.sort_values('win_rate', ascending=False)[['total_pnl', 'sharpe_ratio', 'max_drawdown', 'win_rate']].head())

# # Find optimal stop loss based on Sharpe ratio
# optimal_idx = results_df['sharpe_ratio'].idxmax()
# optimal_sl_bps = optimal_idx
# optimal_sl_pct = optimal_sl_bps / 10000

# print(f"\n{'='*60}")
# print(f"OPTIMAL STOP LOSS (by Sharpe Ratio): {optimal_sl_bps:.1f} bps ({optimal_sl_pct:.4f}%)")
# print(f"Total PnL: {results_df.loc[optimal_idx, 'total_pnl']:,.2f} {analysis_currency_code}")
# print(f"Sharpe Ratio: {results_df.loc[optimal_idx, 'sharpe_ratio']:.3f}")
# print(f"Max Drawdown: {results_df.loc[optimal_idx, 'max_drawdown']:.2%}")
# print(f"Win Rate: {results_df.loc[optimal_idx, 'win_rate']:.2%}")
# print(f"{'='*60}")


In [21]:
# # Visualize optimization results
# from plotly.subplots import make_subplots

# fig = make_subplots(
#     rows=2, cols=2,
#     subplot_titles=('Total PnL vs Stop Loss', 'Sharpe Ratio vs Stop Loss', 
#                     'Max Drawdown vs Stop Loss', 'Win Rate vs Stop Loss'),
#     vertical_spacing=0.12,
#     horizontal_spacing=0.10
# )

# # Total PnL
# fig.add_trace(
#     go.Scatter(x=results_df.index, y=results_df['total_pnl'], 
#                mode='lines+markers', name='Total PnL',
#                line=dict(color='green', width=2)),
#     row=1, col=1
# )
# fig.add_vline(x=optimal_sl_bps, line_dash="dash", line_color="red", 
#               annotation_text=f"Optimal: {optimal_sl_bps:.1f} bps",
#               row=1, col=1)

# # Sharpe Ratio
# fig.add_trace(
#     go.Scatter(x=results_df.index, y=results_df['sharpe_ratio'], 
#                mode='lines+markers', name='Sharpe Ratio',
#                line=dict(color='blue', width=2)),
#     row=1, col=2
# )
# fig.add_vline(x=optimal_sl_bps, line_dash="dash", line_color="red",
#               row=1, col=2)

# # Max Drawdown
# fig.add_trace(
#     go.Scatter(x=results_df.index, y=results_df['max_drawdown'] * 100, 
#                mode='lines+markers', name='Max Drawdown',
#                line=dict(color='red', width=2)),
#     row=2, col=1
# )
# fig.add_vline(x=optimal_sl_bps, line_dash="dash", line_color="red",
#               row=2, col=1)

# # Win Rate
# fig.add_trace(
#     go.Scatter(x=results_df.index, y=results_df['win_rate'] * 100, 
#                mode='lines+markers', name='Win Rate',
#                line=dict(color='purple', width=2)),
#     row=2, col=2
# )
# fig.add_vline(x=optimal_sl_bps, line_dash="dash", line_color="red",
#               row=2, col=2)

# # Update axes labels
# fig.update_xaxes(title_text="Stop Loss (bps)", row=1, col=1)
# fig.update_xaxes(title_text="Stop Loss (bps)", row=1, col=2)
# fig.update_xaxes(title_text="Stop Loss (bps)", row=2, col=1)
# fig.update_xaxes(title_text="Stop Loss (bps)", row=2, col=2)

# fig.update_yaxes(title_text=f"PnL ({analysis_currency_code})", row=1, col=1)
# fig.update_yaxes(title_text="Sharpe Ratio", row=1, col=2)
# fig.update_yaxes(title_text="Max Drawdown (%)", row=2, col=1)
# fig.update_yaxes(title_text="Win Rate (%)", row=2, col=2)

# fig.update_layout(
#     title_text="Stop Loss Optimization Results",
#     showlegend=False,
#     height=700,
#     template="plotly_white"
# )

# fig.show()


In [22]:
analysis_currency = instrument.quote_currency
analysis_currency_code = analysis_currency.code


def analyze_engine(engine, label):
    analyzer = engine.portfolio.analyzer
    pnl_stats = analyzer.get_performance_stats_pnls(currency=analysis_currency)
    returns_stats = analyzer.get_performance_stats_returns()
    general_stats = analyzer.get_performance_stats_general()

    # Use per-trade return series to reconstruct an equity curve.
    returns = analyzer.returns().sort_index()
    equity_curve = (1.0 + returns).cumprod() * starting_balance
    equity_curve.name = label

    return pnl_stats, returns_stats, general_stats, equity_curve


stats_pnls_base, stats_returns_base, stats_general_base, equity_base = analyze_engine(engine, "Gotobi")
stats_pnls_sl, stats_returns_sl, stats_general_sl, equity_sl = analyze_engine(engine_sl, "Gotobi + Stop Loss")

stats_pnls = pd.DataFrame([stats_pnls_base, stats_pnls_sl], index=["Gotobi", "Gotobi + Stop Loss"])
stats_returns = pd.DataFrame([stats_returns_base, stats_returns_sl], index=["Gotobi", "Gotobi + Stop Loss"])
stats_general = pd.DataFrame([stats_general_base, stats_general_sl], index=["Gotobi", "Gotobi + Stop Loss"])

equity_curves = pd.concat([equity_base, equity_sl], axis=1).dropna(how="all")
if equity_curves.index.tz is not None:
    equity_curves.index = equity_curves.index.tz_convert("Asia/Tokyo")

fig = go.Figure()
for column in equity_curves.columns:
    fig.add_trace(
        go.Scatter(
            x=equity_curves.index,
            y=equity_curves[column],
            mode="lines",
            name=column,
        )
    )

fig.update_layout(
    title=f"Gotobi Equity Curves ({analysis_currency_code})",
    xaxis_title="Time (Asia/Tokyo)",
    yaxis_title=f"Equity ({analysis_currency_code})",
    template="plotly_white",
)

fig.show()


In [23]:
summary = pd.DataFrame(
    {
        "Final Equity": equity_curves.ffill().iloc[-1],
        "Total PnL": stats_pnls["PnL (total)"],
    }
)
summary


,Final Equity,Total PnL
Gotobi,151637.597777,82.0
Gotobi + Stop Loss,151637.597777,82.0
